# Genetic algorithm-based heuristic for feature selection in credit risk assessment

First we import libraries ,we will be needed through this notebook : 

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold, cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from numpy.random import rand as random
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import adam
from keras.optimizers import SGD

## Configes

In [16]:
config = { 
    'maxFeatureNum': 12,
    'minFeatureNum': 5,
    'populationSize': 50,
    'initProb': .5,
    'crossoverProb': .9
}

In [17]:
def accuracy(predictions, labels):
#     print(predictions)
#     print(labels)
    return (100.0 * np.sum(predictions.reshape(-1) == labels.reshape(-1))
          / predictions.shape[0])

## Implementation of Neuralnetwork architecture

In [18]:
def tens_train(X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
    """Train one layer feedforward neural network
    Args :
       X : training data
       Y : training label
       hiddenNum : number of hidden units of hidden layer
       trainCycles : number of training cycles
       lr : learning rate of nueral network
       m: momentum of neural network
    Returns :
       'float' accuracy
    """
    
    hidden_size = hiddenNum
    batch_size = len(X)
    attr_size = X.shape[1]
    num_steps = 450
    num_labels = 1
    graph = tf.Graph()
    
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, attr_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#         tf_valid_dataset = tf.constant(valid_dataset)
#         tf_test_dataset = tf.constant(test_dataset)

        # layer1
        weights_1 = tf.Variable(
        tf.truncated_normal([attr_size, hidden_size]))
        biases_1 = tf.Variable(tf.zeros([hidden_size]))
        layer_1 = tf.nn.sigmoid(tf.matmul(tf_train_dataset, weights_1) + biases_1)

        # layer2  
        weights_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        # Training computation.
        layer_2 = tf.matmul(layer_1,weights_2) + biases_2
  
        # calculating Loss
        loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_train_labels, logits=layer_2))

        # Optimizer.
        optimizer = tf.train.MomentumOptimizer(lr,m).minimize(loss)
        train_prediction = tf.nn.sigmoid(layer_2)
        
        
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")

        
        for step in range(trainCycles):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (X.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = X[offset:(offset + batch_size), :]
            batch_labels = Y[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            

In [19]:
def tens_train(X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
    """Train one layer feedforward neural network
    Args :
       X : training data
       Y : training label
       hiddenNum : number of hidden units of hidden layer
       trainCycles : number of training cycles
       lr : learning rate of nueral network
       m: momentum of neural network
    Returns :
       'float' accuracy
    """
    
    hidden_size = hiddenNum
    batch_size = len(X)
    attr_size = X.shape[1]
    num_steps = 450
    num_labels = Y.shape[1]
    graph = tf.Graph()
    
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, attr_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#         tf_valid_dataset = tf.constant(valid_dataset)
#         tf_test_dataset = tf.constant(test_dataset)

        # layer1
        weights_1 = tf.Variable(
        tf.truncated_normal([attr_size, hidden_size]))
        biases_1 = tf.Variable(tf.zeros([hidden_size]))
        layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

        # layer2  
        weights_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        # Training computation.
        layer_2 = tf.matmul(layer_1,weights_2) + biases_2
  
        # calculating Loss
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=layer_2))

        # Optimizer.
        optimizer = tf.train.MomentumOptimizer(lr,m).minimize(loss)
        train_prediction = tf.nn.sigmoid(layer_2)
        
        
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        feed_dict = {tf_train_dataset : X, tf_train_labels : Y}
        _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        print("sd sMinibatch accuracy: %.1f%%" % accuracy(predictions, Y))
        for step in range(trainCycles):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
#             offset = (step * batch_size) % (X.shape[0] - batch_size)
            # Generate a minibatch.
#             batch_data = X[offset:(offset + batch_size), :]
#             batch_labels = Y[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : X, tf_train_labels : Y}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, Y))
            

In [20]:
# x_train = pd.read_excel('GermanCreditInput.xls',header=None)
# x_train[[0,1,2,3,4,6,9,10,11,12,13,14,17,19,20,21,22,24,27,29]]
# y_train = pd.read_excel('GermanCreditOutputClass1columnknn.xls',header=None)
# y_train = (np.arange(2) == np.array(y_train)[:]).astype(np.float32)
# tens_train(np.array(x_train),np.array(y_train),90,1000,.001,.1)
# x_train[]
# print(x_train[x_train.index[[False]*29 + [True]]])

In [21]:
np.array([[2],[4],[6]])[2,None]

array([[6]])

In [56]:
(lambda : "hello world")()

'hello world'

In [65]:
def keras_model(input_dim,hiddenNum=40,lr=.1,m=.5) :
    model = Sequential()
    model.add(Dense(hiddenNum, input_dim=input_dim, kernel_initializer='normal', activation='selu'))
    # model.add(Dense(hiddenNum, kernel_initializer='normal', activation='selu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    sgd = SGD(lr=lr, momentum=m)
    ADAM = adam(lr=lr)
    # loss could be "mse" too
    model.compile(loss='binary_crossentropy',metrics=['accuracy','binary_accuracy'],optimizer=sgd)
    return lambda: model

In [66]:
def simple_train(X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
    """Train one layer feedforward neural network
    Args :
       X : training data
       Y : training label
       hiddenNum : number of hidden units of hidden layer
       trainCycles : number of training cycles
       lr : learning rate of nueral network
       m: momentum of neural network
    Returns :
       'float' accuracy
    """
    model = keras_model(X.shape[1],hiddenNum,lr,m)
    classifier = KerasClassifier(build_fn=keras_model(X.shape[1],hiddenNum,lr,m), 
                                 epochs=450,batch_size=len(X),verbose=0)
#     for i in range(trainCycles) :
#         combined = list(zip(X, Y))
#         np.random.shuffle(combined)
#         X[:], Y[:] = zip(*combined)
#         model.fit(X,Y,batch_size=len(X))
    return cross_val_score(classifier, X, Y, cv=10)

In [67]:
x_train = pd.read_excel('GermanCreditInput.xls',header=None)
y_train = pd.read_excel('GermanCreditOutputClass1columnknn.xls',header=None)
# y_train = (np.arange(2) == np.array(y_train)[:]).astype(np.float32)
# X = np.append(x_train[y_train[0] == 1][:300], x_train[y_train[0] == 0][:300],axis=0)
# Y = np.append(y_train[y_train[0] == 1][:300], y_train[y_train[0] == 0][:300],axis=0)
# combined = list(zip(X, Y))
# np.random.shuffle(combined)
X, Y = x_train,y_train
# np.sum((np.array(y_train).reshape(-1) == 1).argmax())
# (np.array(y_train).reshape(-1) == 1)
simple_train(np.array(X),np.array(Y),30,600,.3,.9)
# x_train[y_train == 1]

array([ 0.75      ,  0.68000001,  0.76999998,  0.72000003,  0.72000003,
        0.58999997,  0.69999999,  0.68000001,  0.70999998,  0.68000001])

In [26]:
class Chromosome : 
    chromosomes = {} # Every Computed Chromosome will be save in this chromosomes
    def __init__(self,genNum,trueProb) :
        self.chrom = np.array([True if random() > trueProb else False for _ in range(genNum)])
        self.fitness = None

    def __iter__(self) :
        return np.nditer(self.chrom)
        
    def mutate(self,prob) :
        self.chrom = np.vectorize(lambda x:x if random() > prob else not x)(self.chrom)
        
    def calculateFitness(self,X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
        """Train one layer feedforward neural network
        Args :
           X : training data
           Y : training label
           hiddenNum : number of hidden units of hidden layer
           trainCycles : number of training cycles
           lr : learning rate of nueral network
           m: momentum of neural network
        Returns :
           'float' accuracy
        """
        string_arr = ''.join(map(str, 1*self.chrom))
        if string_arr not in chromosomes :
            train_data = X[X.index[self.chrom]]
            model = Sequential()
            model.add(Dense(hiddenNum, input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
            ADAM = adam(lr=lr)
            # loss could be "mse" too
            model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=ADAM)
            model.fit(train_data,Y,epochs=trainCycles)
            chromosomes[string_arr] = model.accuracy # hashtable
        return chromosomes[string_arr]

In [335]:
class Population :
    def __init__(self,popSize,genNum,trueProb,X,Y) :
        self.pop = np.array([])
        self.popSize = popSize
        self.genNum = genNum
        self.trueProb = trueProb
        self.X = X
        self.Y = Y
        for _ in range(popSize) : # initializing chromosome
            np.append(self.pop,Chromosome(genNum,trueProb))
        
    def __iter__(self) :
        return np.nditer(self.pop)
    
    def mutate(self,prob) :
        for chrom in self :
            chrom.mutate(prob)
    
    def crossover(self,prob) :
        newPop = Population(self.popSize,self.genNum,self.trueProb,self.X,self.Y)
        for i in range(len(self.pop)) :
            rand_prob = np.random.random()
            if rand_prob > prob :
                first = self.pop[np.random.randint(len(self.pop))].chrom
                second = self.pop[np.random.randint(len(self.pop))].chrom
                point = random.randint(len(first))
                newPop.pop[i].chrom = np.append(first[:point] + second[point:])
            else :
                newPop.pop[i].chrom = np.append(first[:point] + second[point:])
        return newPop

In [372]:
x_train = pd.read_excel('GermanCreditInput.xls',header=None)
y_train = pd.read_excel('GermanCreditOutputClass1columnknn.xls',header=None)
k_fold = KFold(n_splits=10)
for train_indices, test_indices in k_fold.split(x_train):
    print(x_train.iloc[train_indices])

     0   1   2   3   4   5   6   7   8      9  ...  20  21  22  23  24  25  \
100   3  24   2   1   0   0   0   0   0   1469 ...   0  41   0   1   0   1   
101   1  36   2   0   0   0   1   0   0   2323 ...   0  24   0   1   0   1   
102   3   6   3   0   0   0   1   0   0    932 ...   0  24   0   0   1   1   
103   1   9   4   0   0   1   0   0   0   1919 ...   0  35   0   1   0   1   
104   3  12   2   0   1   0   0   0   0   2445 ...   0  26   0   1   0   1   
105   1  24   4   0   0   0   0   0   0  11938 ...   0  39   0   0   1   2   
106   3  18   1   1   0   0   0   0   0   6458 ...   1  39   1   0   1   2   
107   1  12   2   1   0   0   0   0   0   6078 ...   0  32   0   0   1   1   
108   0  24   2   0   0   1   0   0   0   7721 ...   0  30   0   0   1   1   
109   1  14   2   0   0   0   0   0   1   1410 ...   0  35   0   0   1   1   
110   1   6   3   0   0   0   0   0   1   1449 ...   0  31   1   0   1   2   
111   2  15   2   0   0   0   0   1   0    392 ...   0  23   0  

In [20]:
class GeneticAlgorithm :
    def __init__(self,initPopSize,genNum,trueProb) :
        self.bestChrom = None
        
    def run(self,mProb,cProb) :
        